In [ ]:
from snp_query_box.dsnp_transform import dsnpTransform

In [ ]:
#for dev purpose
import getpass
import os
USERNAME = os.getenv('USERNAME').lower()
PASSWORD = getpass.getpass()
new_pull = True

In [ ]:
import os
import pandas as pd
import numpy as np

import datetime as dt
import time
import warnings
from dateutil.relativedelta import relativedelta


from snp_query_box import DsnpHelperFunction, snpQueries, populDashQueries

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
print(f"User: {USERNAME} Started creating pull_utilization, pull_er_visit, pull_ip_visit_from_hedis and *_12month in the shared storage")

In [ ]:
start_time = time.time()

In [ ]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print(first_date_of_last_month_str)

pulled_data_date = last_month
comp_report_data_month = last_month
hra_report_data_month = last_month
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
one_year_rolling_back_date

In [ ]:
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
reporting_year=2025

In [ ]:
# if transition time first month...
if today_str.split('-')[1] =='01':
    first_date_of_year = str(int(first_date_of_year.split('-')[0])-1) + \
        '-01-01'
    print('Trainsition time, first date of year is updated by previous year:' + first_date_of_year)
    #first_date_of_year = '2024-01-01'

In [ ]:
#locate data dump folder
if new_pull:
    pull_utilization_12month = populDashQueries.pull_utilization(USERNAME, PASSWORD, start_date=one_year_rolling_back_date, end_date=reporting_end_date, reporting_year=reporting_year)
    pull_er_visit_12month = populDashQueries.pull_er_visit(USERNAME, PASSWORD, start_date=one_year_rolling_back_date, end_date=reporting_end_date, reporting_year=reporting_year)
    pull_ip_visit_12month = populDashQueries.pull_ip_visit_from_hedis(start_date=one_year_rolling_back_date, end_date=reporting_end_date)

    pull_utilization_12month.to_parquet(f"{output_path}/pull_utilization_12month.parquet")
    pull_er_visit_12month.to_parquet(f"{output_path}/pull_er_visit_12month.parquet")
    pull_ip_visit_12month.to_parquet(f"{output_path}/pull_ip_visit_12month.parquet")
else:
    try:
        pull_utilization_12month = pd.read_parquet(f"{output_path}/pull_utilization_12month.parquet")
        pull_er_visit_12month = pd.read_parquet(f"{output_path}/pull_er_visit_12month.parquet")
        pull_ip_visit_12month = pd.read_parquet(f"{output_path}/pull_ip_visit_12month.parquet")

    except:
        pull_utilization_12month = populDashQueries.pull_utilization(USERNAME, PASSWORD, start_date=one_year_rolling_back_date, end_date=reporting_end_date, reporting_year=reporting_year)
        pull_er_visit_12month = populDashQueries.pull_er_visit(USERNAME, PASSWORD, start_date=one_year_rolling_back_date, end_date=reporting_end_date, reporting_year=reporting_year)
        pull_ip_visit_12month = populDashQueries.pull_ip_visit_from_hedis(start_date=one_year_rolling_back_date, end_date=reporting_end_date)

        pull_utilization_12month.to_parquet(f"{output_path}/pull_utilization_12month.parquet")
        pull_er_visit_12month.to_parquet(f"{output_path}/pull_er_visit_12month.parquet")
        pull_ip_visit_12month.to_parquet(f"{output_path}/pull_ip_visit_12month.parquet")

In [ ]:
pull_utilization_12month.head()

In [ ]:
pull_utilization_ytd = populDashQueries.pull_utilization(USERNAME, PASSWORD, start_date=first_date_of_year, end_date=reporting_end_date, reporting_year=reporting_year)


In [ ]:
pull_er_visit_ytd = pull_er_visit_12month[pull_er_visit_12month["srv_start_dt"]>=np.datetime64(first_date_of_year)]
pull_ip_visit_ytd = pull_ip_visit_12month[pull_ip_visit_12month["AdmitDate"]>=np.datetime64(first_date_of_year)]

In [ ]:
#Prepare call log data, one year back
utilization_df_ytd = dsnpTransform.get_utilization(pull_utilization_ytd, pull_ip_visit_ytd, pull_er_visit_ytd)
utilization_df_12month = dsnpTransform.get_utilization(pull_utilization_12month, pull_ip_visit_12month, pull_er_visit_12month)

In [ ]:
#save it for monthly aggregation
utilization_df_ytd.to_parquet(f"{output_path}/utilization_df_ytd.parquet")
utilization_df_12month.to_parquet(f"{output_path}/utilization_df_12month.parquet")

In [ ]:
utilization_df_ytd.head()

In [ ]:
print("Process time -- %s seconds" % (time.time() - start_time))